In [1]:
from stackapi import StackAPI
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
import os
from tqdm import tqdm

In [2]:
so_website = StackAPI('stackoverflow')

In [3]:
so_website.max_pages = 20

In [4]:
scrape_date_start = datetime(2018, 1, 1)
scrape_date_finish = datetime(2018, 8, 1)

In [5]:
date_pairs = [(
    scrape_date_start + timedelta(days = x), scrape_date_start + timedelta(days = x + 1)
) for x in range((scrape_date_finish - scrape_date_start).days + 1)]

In [6]:
all_questions = []

In [7]:
date_itr = tqdm(date_pairs)
for (dfrom, dto) in date_itr:
    question = so_website.fetch(
        'questions',
        fromdate = dfrom,
        todate = dto,
        tagged = "rust",
        sort = 'creation'
    )
    all_questions.extend(question['items'])
    date_itr.set_postfix({"scraped questions count": len(all_questions)})
    sleep(1)

100%|██████████| 213/213 [05:30<00:00,  1.56s/it, scraped questions count=1584]


In [8]:
questions_df = pd.DataFrame(
    [(q['question_id'], q['view_count'], q['answer_count'], q['creation_date'], q['title'], q['owner']['reputation'], ",".join(q["tags"])) for q in all_questions if 'reputation' in q['owner']],
    columns=['question_id', 'view_count', 'answer_count', 'creation_date', 'title', 'reputation', 'tags']
)

In [9]:
questions_df.head()

,question_id,view_count,answer_count,creation_date,title,reputation,tags
0,48053933,995,1,1514847365,How to check if a directory exists and create ...,2176,"directory,rust"
1,48050155,101,1,1514814164,Is it possible to get the full &quot;namespace...,1128,rust
2,48068334,942,1,1514929904,Taking multiple values in an argument in Clap,95647,"rust,clap"
3,48067874,816,2,1514927374,How do I inspect local variables or use the wa...,2930,"rust,visual-studio-code,lldb"
4,48067556,80,2,1514925630,What is the difference between char::is_digit ...,711,"unicode,utf-8,rust"


In [10]:
len(questions_df)

1567

In [11]:
os.makedirs("../data/", exist_ok=True)

In [12]:
questions_df.to_feather("../data/00-raw-questions.feather")